In [24]:
import pandas as pd
import plotly.express as px
from src import config
from tqdm import tqdm
from src.utils.llm.openai_client import get_proposer
predictions = pd.read_parquet("output/predictions.parquet")

In [25]:
predictions

,vote_id,type,title,drucksache_id,beschlussempfehlung,summary,summary_embedding,date,party,reasoning,prediction,ground_truth,category,is_governing,bundestag,vote_correct,proposers,is_own_proposal
0,20250318_3,Gesetzentwurf,Gesetzentwurf der Fraktionen der SPD und CDU/C...,20/15096,None,Abgestimmt wird über die Einrichtung einer ver...,"[0.009540573693811893, 0.03497336432337761, 0....",2025-03-18,AfD,{'context': '- Arbeitslosenversicherung reform...,Ablehnung,Ablehnung,"Verteidigung & Sicherheit - Militär, Verteidig...",False,20,True,"[SPD, Union]",False
1,20250318_2,Änderungsantrag,Änderungsantrag der Fraktion der FDP zu der zw...,20/15120,None,"Der Bundestag soll beschließen, das Grundgeset...","[0.011175020597875118, 0.03776725009083748, 0....",2025-03-18,AfD,{'context': '- Innere Sicherheit stärken: Bund...,Ablehnung,Ablehnung,"Verteidigung & Sicherheit - Militär, Verteidig...",False,20,True,[FDP],False
2,20250318_1,Gesetzentwurf,Gesetzentwurf der Fraktion der FDP Entwurf ein...,20/15099,None,Im Bundestag wird über die Erweiterung des Son...,"[0.012960884720087051, 0.037686433643102646, 0...",2025-03-18,AfD,{'context': '- Innere Sicherheit stärken: Bund...,Ablehnung,Ablehnung,"Verteidigung & Sicherheit - Militär, Verteidig...",False,20,True,[FDP],False
3,20250131_1,Gesetzentwurf,Gesetzentwurf der Fraktion der CDU/CSU Entwurf...,20/12804,None,Das Gesetz soll das Ziel der Begrenzung der Zu...,"[0.04290461167693138, 0.07238075137138367, 0.0...",2025-01-31,AfD,{'context': '- Rückführungsoffensive: Ausreise...,Annahme,Annahme,"Inneres & Migration - Innere Sicherheit, öffen...",False,20,True,[Union],False
4,20241219_1,Gesetzentwurf,"Gesetzentwurf der Fraktionen SPD, CDU/CSU, BÜN...",20/12977,None,Im Bundestag wird über einen Gesetzentwurf abg...,"[-0.013081829994916916, 0.0215305108577013, 0....",2024-12-19,AfD,{'context': '- Demokratie und Rechtsstaat - ...,Annahme,Ablehnung,"Justiz & Verbraucherschutz - Rechtsprechung, G...",False,20,False,"[SPD, Union, DIE_GRÜNEN, FDP]",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7228,20121213_4,Antrag,Antrag der Bundesregierung Fortsetzung des Ein...,17/11466,Annahme,Der Bundestag stimmt der Fortsetzung des Einsa...,"[-0.00255092466250062, 0.07303918898105621, 0....",2012-12-13,Union,{'context': '- Anpassung des Jugendstrafrechts...,Annahme,Annahme,"Verteidigung & Sicherheit - Militär, Verteidig...",True,17,True,[Bundesregierung],True
7229,20121213_2,Antrag,"Antrag der Abgeordneten Johanna Voß, Ulla Lötz...",17/11328,Ablehnung,Abgestimmt wird über ein Gesetz zur vollständi...,"[-0.004911317024379969, 0.06889131665229797, 0...",2012-12-13,Union,"{'context': '- Bekämpfung von Protektionismus,...",Annahme,Ablehnung,"Umwelt, Klima & Naturschutz - Umweltschutz, Kl...",True,17,False,[DIE_LINKE],False
7230,20121213_2,Antrag,"Antrag der Abgeordneten Johanna Voß, Ulla Lötz...",17/11328,Ablehnung,Abgestimmt wird über ein Gesetz zur vollständi...,"[-0.004911317024379969, 0.06889131665229797, 0...",2012-12-13,Union,"{'context': '- Bekämpfung von Protektionismus,...",Annahme,Ablehnung,"Umwelt, Klima & Naturschutz - Umweltschutz, Kl...",True,17,False,[DIE_GRÜNEN],False
7231,20121213_2,Antrag,"Antrag der Abgeordneten Oliver Krischer, Nicol...",17/11213,Ablehnung,Der Bundestag soll über ein Moratorium für die...,"[0.003851190907880664, 0.06292638927698135, -0...",2012-12-13,Union,{'context': '- Stärkung der Investitionsfähigk...,Annahme,Ablehnung,"Umwelt, Klima & Naturschutz - Umweltschutz, Kl...",True,17,False,[DIE_LINKE],False
